In [92]:
import tensorflow as tf

import numpy as np
import os
import time

In [93]:
path_to_file = 'dataset.txt'

In [94]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 13810631 characters


In [95]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

102 unique characters


In [96]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [97]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [98]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[61, 62, 63, 64, 65, 66, 67], [84, 85, 86]]>

In [99]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [100]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [101]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [102]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [103]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(13810631,), dtype=int64, numpy=array([51, 68, 69, ..., 65,  2,  1], dtype=int64)>

In [104]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [105]:
seq_length = 1000

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [106]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [107]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [108]:
dataset = sequences.map(split_input_target)

In [109]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 1000), dtype=tf.int64, name=None), TensorSpec(shape=(64, 1000), dtype=tf.int64, name=None))>

In [110]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [111]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [112]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [113]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 1000, 103) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [114]:
model.summary()

Model: "my_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     multiple                  26368     
                                                                 
 gru_4 (GRU)                 multiple                  3938304   
                                                                 
 dense_4 (Dense)             multiple                  105575    
                                                                 
Total params: 4,070,247
Trainable params: 4,070,247
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [115]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [116]:
sampled_indices

array([ 67,  56,  48,  25, 100,  99,  78,  93,  18,  72,  99,  18,  50,
        98,  71,  16,  66,  69,  44,  84,  86,  53,  69,  78,  13,  34,
        15,  61,  91,  39,  45,  52,  93,  38,  13,  67,  83,  59,  24,
         5,  39,   3,  79,   7,  54,  87,   4,  14,  23,  86,   6,  28,
        19,  26,  95,   4,  62,   7,  14,   9,   1,   2,  23,  82,  40,
        46,  27,  65,  43,  55,  65,  58,  14,  99,  39,  87,  46,  60,
        65,  60,  99,  94,  87,  13,  79,  51,  19,  85,  32,   0,  38,
        81,  88,  39,  43,  67,  27,  64,  23,  48,  89,  20,  95,  25,
        79,  12,  92,  93,  85,  75,  99,  78,  97,   5,  35,  49,  51,
        69,  54, 100,   3,  18,  31,  95,  52,  38,  28,   9,  45,  11,
        38,  47,  79,  25,   4,  45,  87,  22,  99,  71,  97,  39,  65,
         7,  24,  91,  89,  99,  23,  92,  50,  39,  47,   7,  61,  41,
         5,   2,  73,  91,  18,  37,  64,  96,  56,  85,  47,  49,  87,
        47,  25,  55,  32,  55,  49,   1,  36,  75,  29,  22,  4

Decode these to see the text predicted by this untrained model:

In [117]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"ute lunch depending yet i go to the\r\nbathroom for four [\xc2\xa0__\xc2\xa0] minutes and\r\nall of a sudden you gotta know exactly\r\nwhere i'm at i don't get to leave for 15\r\nminutes if i go live for eight hours i\r\ndon't get to do a 30 minute lunch while\r\ni'm working my ass off\r\nputting my pedal to the goddamn metal\r\ngrinding churning out these dollar bills\r\nfor daddy bezos to make more money i\r\ncan't get five minutes in the bathroom\r\nwithout a [\xc2\xa0__\xc2\xa0] being like\r\nprobably jerking off\r\na fellow viewer in the chat room made a\r\nvalid point the point was that ludwig\r\nmay be taking long due to the high\r\nlevels of testosterone in his system\r\nwith that being said it was connecting\r\nthe dots from there\r\neverybody knows every time you jerk off\r\nyou lose your testosterone that's right\r\nif you guys don't know floyd mayweather\r\nis a virgin he has gotten a record zero\r\n[\xc2\xa0__\xc2\xa0] in a record 49 wins and zero\r\nlosses\r\nth

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [118]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [119]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 1000, 103)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.633491, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [120]:
tf.exp(example_batch_mean_loss).numpy()

102.87257

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [121]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [122]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [123]:
EPOCHS = 20

In [124]:
# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [125]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [126]:
# one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [127]:
# start = time.time()
# states = None
# next_char = tf.constant(['ROMEO:'])
# result = [next_char]

# for n in range(1000):
#   next_char, states = one_step_model.generate_one_step(next_char, states=states)
#   result.append(next_char)

# result = tf.strings.join(result)
# end = time.time()
# print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
# print('\nRun time:', end - start)

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [128]:
# start = time.time()
# states = None
# next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
# result = [next_char]

# for n in range(1000):
#   next_char, states = one_step_model.generate_one_step(next_char, states=states)
#   result.append(next_char)

# result = tf.strings.join(result)
# end = time.time()
# print(result, '\n\n' + '_'*80)
# print('\nRun time:', end - start)

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [129]:
# tf.saved_model.save(one_step_model, 'one_step')
# one_step_reloaded = tf.saved_model.load('one_step')

In [130]:
# states = None
# next_char = tf.constant(['ROMEO:'])
# result = [next_char]

# for n in range(100):
#   next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
#   result.append(next_char)

# print(tf.strings.join(result)[0].numpy().decode("utf-8"))

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [131]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [132]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [133]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [134]:
model.fit(dataset, epochs=50)

Epoch 1/50
215/215 [==============================] - 90s 395ms/step - loss: 2.3896
Epoch 2/50
215/215 [==============================] - 89s 395ms/step - loss: 1.6145
Epoch 3/50
215/215 [==============================] - 90s 397ms/step - loss: 1.3749
Epoch 4/50
215/215 [==============================] - 89s 396ms/step - loss: 1.2765
Epoch 5/50
215/215 [==============================] - 89s 397ms/step - loss: 1.2235
Epoch 6/50
215/215 [==============================] - 89s 397ms/step - loss: 1.1896
Epoch 7/50
215/215 [==============================] - 89s 396ms/step - loss: 1.1652
Epoch 8/50
215/215 [==============================] - 89s 396ms/step - loss: 1.1459
Epoch 9/50
215/215 [==============================] - 89s 395ms/step - loss: 1.1304
Epoch 10/50
215/215 [==============================] - 89s 395ms/step - loss: 1.1172
Epoch 11/50
215/215 [==============================] - 89s 395ms/step - loss: 1.1058
Epoch 12/50
215/215 [==============================] - 89s 395ms/step - lo

Or if you need more control, you can write your own complete custom training loop:

In [138]:
tf.saved_model.save(model, 'model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [139]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [140]:
states = None
next_char = tf.constant(["Boys today the plan is simple"])
result = [next_char]

for n in range(5000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Boys today the plan is simple
sauce and my block was innovading
monkeys I spent $5,000 Jesus
Christmas pleasure I've eaten our
my proffscares twice as well jackettly
situation she will get the same thing
you can't see your friends for the man
who command heat it makes you look like
you two for four right yeah for sure
I'm yound eyes balling it should not
clock it 2018 likes to
whatever you are agt so it's balage
every chart watch the Content swipe d
I was like wait a minute
in bucks it kind of, fits I think we ate
the milosional tonight for once I have
leftovers what a good thing and so now I
flop both of us look all right let's buy
tax evasion say what the [ __ ]
it is
is this dream look anime style boy
banning the idea better what are they
look far feel in the goddamn russian oh
they're [ __ ] jake it's your guts it
just it's not for washing ninja your
turn it's a piece of [ __ ] are you
kidding me
oh stick you [ __ ] oh my god it's so
ad something depending too far
what are you on M

In [141]:
tf.saved_model.save(one_step_model, 'one_step_custom')
one_step_reloaded = tf.saved_model.load('one_step_custom')

INFO:tensorflow:Assets written to: one_step_custom\assets


INFO:tensorflow:Assets written to: one_step_custom\assets
